In [1]:
import os
import glob
import re
from pythainlp.tokenize import word_tokenize, isthai
from pythainlp.tag import pos_tag
from pythainlp.corpus import stopwords
import heapq
from gensim.models import Word2Vec
from pythainlp.word_vector import thai2vec

C:\Users\Watchanan\PycharmProjects\thai-question-answering\venv\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
thai_stopwords = stopwords.words('thai')

examples_path = 'Examples\\'
question_list_filename = examples_path + 'question_list.txt'
expected_answer_list_filename = examples_path + 'expected_ans.txt'
sources_dir_name = examples_path + 'Sources\\'

numbers = re.compile(r'(\d+)')
def numerical_sort(value):
    parts = numbers.split(value)
    parts[1::2] = map(int, parts[1::2])
    return parts

In [3]:
document_list = []
for source_filename in sorted(glob.glob(os.path.join(sources_dir_name, '*.txt')), key=numerical_sort):
    with open(source_filename , encoding='utf-8') as f:
        document = []
        for line in f:
            document.append(line.rstrip('\n'))
        document_list.append(document)

question_list = []
with open(question_list_filename, encoding='utf-8') as f:
    for line in f:
        question_list.append([n for n in line.strip().split('::')])
print("Questions = ", question_list)

expected_answer_list = []
with open(expected_answer_list_filename, encoding='utf-8') as f:
    for line in f:
        expected_answer_list.append([n for n in line.strip().split('::')])
print("Expected answers = ", expected_answer_list)

Questions =  [['\ufeffQ1', 'ข้าวสามารถไว้ตอได้นานเท่าไหร่'], ['Q2', 'จิงโจ้ใช้เวลาตั้งท้องประมาณกี่วัน'], ['Q3', 'ครูจอมทรัพย์จ้างนายสับ วาปีมารับผิดแทนด้วยเงินเท่าไหร่'], ['Q4', 'ปี พ.ศ. 2561 มีหนังสือที่ได้รับรางวัลจากการประกวดหนังสือดีเด่นทั้งหมดกี่เรื่อง'], ['Q5', 'อัตราค่าโดยสารของเรือในคลองแสนแสบลดลงระยะละกี่บาท'], ['Q6', 'จำนวนสัตว์ป่าที่เป็นสัตว์เลี้ยงลูกด้วยนมในจังหวัดจันทบุรีนั้นมีทั้งสิ้นกี่ชนิด'], ['Q7', 'รร.หอวัง มีห้องเรียนพิเศษสำหรับชั้น ม.4 อยู่กี่ห้อง'], ['Q8', 'กุ้งก้ามแดงต้องมีอายุประมาณเท่าไหร่จึงจะสามารถขายเป็นพ่อแม่พันธุ์ได้'], ['Q9', 'ในปี 2557 Uber มีให้บริการให้ทั้งหมดกี่ประเทศ'], ['Q10', 'แพนด้ายักษ์มีหางยาวประมาณเท่าไหร่'], ['Q11', 'ใครเป็นผู้กำกับการแสดงละครเรื่องตะวันยอแสงของช่อง 3  ในปี พ.ศ. 2560'], ['Q12', 'ผู้ใดคือประธานการประชุมคณะกรรมการจัดระบบการจราจรทางบก'], ['Q13', 'ใครเป็นผู้นำข้าวโพดพันธุ์ที่ใช้เลี้ยงสัตว์มาปลูกในประเทศไทย'], ['Q14', 'เพื่อนของมธุสรมีชื่อว่าอะไร'], ['Q15', 'วิหคหลงลมเป็นบทประพันธ์ของใคร'], ['Q16', 'ใครเป็นผู้แทงนายอนันดาเสียชีวิต'

In [4]:
document_data = []

count_doc = 1
for document in document_list:
    # paragraph_struct = {
    #     'original': '',
    #     'tokenized': '',
    #     'tagged': ''
    # }
    for paragraph in document:
        normalized_paragraph = " ".join(paragraph.split())
        normalized_paragraph = normalized_paragraph.replace('"', '')
        tokenized_paragraph = word_tokenize(text=normalized_paragraph, engine='deepcut')
        tokenized_paragraph = [x for x in tokenized_paragraph if x != ' ']
        # try:
        #     tagged_paragraph = pos_tag(tokenized_paragraph, engine='artagger')
        # except:
        #     pass
        # document_data.append(tagged_paragraph)
        # filtered_words = [word for word in tokenized_paragraph if word not in stopwords]
        # document_data.append(filtered_words)
        document_data.append(tokenized_paragraph)
        print(tokenized_paragraph)
    count_doc = count_doc + 1
    # if count_doc > 28:
    #     break
print(document_data)

C:\Users\Watchanan\PycharmProjects\thai-question-answering\venv\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


['\ufeffจังหวัดจันทบุรี', 'มี', 'พื้นที่', 'ป่าไม้', 'มาก', 'ที่สุด', 'ใน', 'ภาค', 'ตะวัน', 'ออก', 'โดย', 'คิด', 'เป็น', '1', 'ใน', '4', 'ของ', 'จำนวน', 'พื้นที่', 'ป่าไม้', 'ทั้ง', 'ภาค', 'อย่าง', 'ไร', 'ก็ตาม', 'หาก', 'เปรียบเทียบ', 'ใน', 'อดีต', 'จะ', 'พบ', 'ว่า', 'จังหวัดจันทบุรี', 'สูญเสีย', 'พื้นที่', 'ป่าไม้', 'ไป', 'เป็น', 'จำนวน', 'มาก', 'เพราะ', 'เดิม', 'ที', 'จังหวัดจันทบุรี', 'มี', 'เนื้อที่', 'ป่าไม้', 'มาก', 'กว่า', 'ร้อย', 'ละ', '50', 'ของ', 'จังหวัด', 'โดย', 'สาเหตุ', 'ของ', 'การ', 'สูญเสีย', 'พื้นที่', 'ป่าไม้', 'ส่วน', 'ใหญ่', 'เกิด', 'จาก', 'การ', 'ลักลอบ', 'ตัด', 'ไม้', 'การ', 'บุกรุก', 'ของ', 'ราษฎร', 'และ', 'การ', 'ขาด', 'การ', 'เข้มงวด', 'กวดขัน', 'ของ', 'เจ้าหน้าที่', 'ใน', 'ปัจจุบัน', 'มี', 'การ', 'ประกาศ', 'ให้', 'พื้นที่', 'ป่าไม้', 'ของ', 'จังหวัดจันทบุรี', 'ขึ้น', 'เป็น', 'อุทยาน', 'แห่งชาติ ', '3', 'แห่ง', 'คือ', 'อุทยานแห่งชาติเขาคิชฌกูฏ', 'อุทยานแห่งชาติเขาสิบ', 'ห้า', 'ชั้น', 'และ', 'อุทยานแห่งชาติน้ำตกพลิ้ว', 'วน', 'อุทยานแห่งชาติ ', '1', 'แห่ง', 'คือ'

['4', '.', 'ข้าวขึ้น', 'น้ำ', 'ข้าว', 'ที่', 'ปลูก', 'ใน', 'นา', 'ที่', 'มี', 'น้ำ', 'ท่วม', 'ขัง', 'ใน', 'ระหว่าง', 'การ', 'เจริญเติบโต', 'ของ', 'ข้าว', 'มี', 'ระดับ', 'น้ำ', 'ลึก', 'ตั้งแต่', '1', '-', '5', 'เมตร', 'เป็น', 'เวลา', 'ไม่', 'น้อย', 'กว่า', '1', 'เดือน', 'ลักษณะ', 'พิเศษ', 'ของ', 'ข้าว', 'ขึ้น', 'น้ำ', 'คือ', 'มี', 'ความ', 'สามารถ', 'ใน', 'การ', 'ยืด', 'ปล้อง', '(', 'internode', 'elongation', 'ability', ')', 'การ', 'แตก', 'แขนง', 'และ', 'ราก', 'ที่', 'ข้อ', 'เหนือ', 'ผิว', 'ดิน', '(', 'upper', 'nodal', 'tillering', 'and', 'rooting', 'ability', ')', 'และ', 'การ', 'ชูรวง', '(', 'kneeing', 'ability', ')']
['5', '.', 'ข้าว', 'น้ำลึก', 'ข้าว', 'ที่', 'ปลูก', 'ใน', 'พื้นที่', 'น้ำ', 'ลึก', 'ระดับ', 'น้ำ', 'ใน', 'นา', 'มาก', 'กว่า', '50', 'เซนติเมตร', 'แต่', 'ไม่', 'เกิน', '100', 'เซนติเมตร']
['6', '.', 'ข้าว', 'ไร่', 'ข้าว', 'ที่', 'ปลูก', 'ใน', 'ที่', 'ดอน', 'หรือ', 'ใน', 'สภาพ', 'ไร่', 'บริเวณ', 'ไหล่', 'เขา', 'หรือ', 'พื้นที่', 'ซึ่ง', 'ไม่', 'มี', 'น้ำ', 'ขัง', 'ไม่', 'มี'

['งาน', 'เทศกาล', 'มาฆปูรมี', 'ศรีปราจีน', 'งาน', 'นี้', 'จัด', 'ขึ้น', 'ที่', 'วัด', 'สระมรกต', 'ใน', 'บริเวณ', 'โบราณสถาน', 'สระมรกต', 'อำเภอศรีมโหสถ', 'เป็น', 'ระยะ', 'เวลา', '3', 'วัน', 'ซึ่ง', 'ตรง', 'กับ', 'วัน', 'มาฆบูชา', 'ของ', 'ทุก', 'ปี', 'ใน', 'งาน', 'กิจกรรม', 'หลัก', 'คือ', 'การ', 'แสดง', 'แสง', 'สี', 'เสียง', 'การ', 'แสดง', 'โขน', 'ตลาด', 'ย้อนยุค', 'รวม', 'ถึง', 'การ', 'เข้า', 'ค่าย', 'พุทธศาสน์', 'และ', 'ปลูก', 'จิต', 'สำนึก', 'ทาง', 'ศาสนา', 'และ', 'วัฒนธรรม', 'ของ', 'นัก', 'เรียน', 'นัก', 'ศึกษา', 'ระดับ', 'มัธยมศึกษา', 'ขึ้น', 'ไป', 'การ', 'ประชุม', 'พระภิกษุ', 'และ', 'พระสังฆาธิการ', 'ใน', 'จังหวัด', 'การ', 'ปฏิบัติ', 'ธรรม', 'ของ', 'พระสงฆ์', 'อุบาสก อุบาสิกา', 'กิจกรรม', 'การ', 'เวียนเทียน', 'รอบ', 'รอย', 'พระพุทธบาทคู่']
['งาน', 'ประเพณี', 'การ', 'แข่ง', 'เรือ', 'ยาว', 'จัด', 'เป็น', 'งาน', 'ประจำ', 'มา', 'ตั้งแต่', 'พ.ศ.', '2528', 'โดย', 'จัด', 'ที่', 'แม่น้ำบางปะกง', 'หรือ', 'แม่น้ำปราจีนบุรี', 'บริเวณ', 'สะพานณรงค์ดำ', 'ริ', 'ถึง', 'หน้า', 'วัดหลวงปรีชากูล', 

['\ufeff', '“', 'แพะ', '”', 'เสียง', 'เรียก', 'หา', 'ความ', 'ยุติธรรม', 'จาก', 'ผู้', 'บริสุทธิ์', 'ที่', 'ต้อง', 'ตก', 'เป็น', 'เหยื่อ', 'ของ', 'กระบวนการ', 'ยุติธรรม', 'โดย', 'ไร้', 'ข้อ', 'ต่อสู้', 'และ', 'หมด', 'โอกาส', 'โต้แย้ง', 'ใด', 'ๆ', 'ทำ', 'ได้', 'เพียง', 'ยอม', 'จำนน', 'เป็น', 'เบี้ยล่าง', 'รับผิด', 'ต่อ', 'ข้อ', 'กล่าวหา', 'ที่', 'ตน', 'เอง', 'ไม่', 'ได้', 'กระทำ', 'ซึ่ง', 'ไม่', 'มี', 'ใคร', 'รับ', 'รู้', 'ความ', 'จริง', 'ได้', 'นอก', 'จาก', 'ตัว', 'ของ', 'เรา', 'เอง', 'และ', 'หนึ่ง', 'ใน', 'แพะ', 'รับบาป', 'ที่', 'กลาย', 'เป็น', 'คดี', 'สั่นคลอน', 'กระบวนการ', 'ยุติธรรม', 'ไล่', 'เรียง', 'ไป', 'ตั้งแต่', 'ชั้น', 'พนักงาน', 'สอบสวน', 'จน', 'ถึง', 'ขบวนการ', 'ตัดสิน', 'ของ', 'ศาล', 'จน', 'มี', 'การ', 'รื้อฟื้น', 'คดี', 'กัน', 'ขึ้น', 'มา', 'ใหม่', 'นั่น', 'ก็', 'คือ', '“', 'คดี', 'ครูจอมทรัพย์', '”', 'หรือ', 'นางจอมทรัพย์', 'แสน', 'เมืองโคตร', 'ที่', 'ถูก', 'ตัดสิน', 'จำ', 'คุก', 'ใน', 'คดี', 'ขับ', 'รถ', 'โดย', 'ประมาท', 'ทำ', 'ให้', 'ผู้', 'อื่น', 'เสีย', 'ชีวิต', 'เป็น

...

['จังหวัดสุรินทร์', 'มี', 'ทรัพยากร', 'ธรรมชาติ', 'ที่', 'สำคัญ', 'คือ', 'ป่าไม้', 'ซึ่ง', 'มี', 'พื้นที่', 'ป่าไม้', 'ทั้ง', 'จังหวัด', 'ประมาณ', '1', ',', '434', ',', '001', 'ไร่', 'ปัจจุบัน', 'มี', 'พื้นที่', 'ป่า', 'สมบูรณ์', 'เหลือ', 'อยู่', 'ประมาณ', '187', ',', '343', 'ไร่', 'จังหวัดสุรินทร์', 'มี', 'พื้นที่', 'ที่', 'ได้', 'รับ', 'การ', 'ประกาศ', 'เป็น', 'เขต', 'ป่า', 'จำนวน', '1', ',', '382', ',', '625', 'ไร่', 'หรือ', 'คิด', 'เป็น', 'ร้อย', 'ละ', '27', '.', '23', 'ของ', 'พื้นที่', 'โดย', 'สามารถ', 'แยก', 'ได้', 'ดัง', 'นี้', 'คือ']
['1', '.', 'เขต', 'ป่า', 'สงวน', 'แห่ง', 'ชาติ', 'จำนวน', '26', 'ป่า']
['2', '.', 'วนอุทยาน', 'จำนวน', '2', 'แห่ง', 'ได้แก่', 'วน', 'อุทยานพนม', 'สวาย', 'อำเภอเมืองสุรินทร์', 'เนื้อที่', '2', ',', '500', 'ไร่', 'และ', 'วนอุทยาน', 'ป่า', 'สนหนองคู', 'อำเภอสังขะ', 'เนื้อที่', '625', 'ไร่']
['3', '.', 'เขต', 'รักษาพันธ์สัตว์', 'ป่า', '1', 'แห่ง', 'คือ', 'เขตรักษาพันธ์สัตว์', 'ป่าห้วยสำราญ', '-', 'ห้วยทับทัน', 'อยู่', 'ใน', 'พื้นที่', 'อำเภอพนมดงรัก', 

['\ufeffบิตคอยน์', '(', 'อังกฤษ', ':', 'Bitcoin', ')', 'เป็น', 'เงินตรา', 'แบบ', 'ดิจิทัล', '(', 'cryptocurrency', ')', 'และ', 'เป็น', 'ระบบ', 'การ', 'ชำระ', 'เงิน', 'ที่', 'ใช้', 'กัน', 'ทั่ว', 'โลก', 'บิตคอยน์', 'เป็น', 'สกุล', 'เงิน', 'ดิจิทัล', 'แรก', 'ที่', 'ใช้', 'ระบบ', 'กระจาย', 'อำนาจ', 'โดย', 'ไม่', 'มี', 'ธนาคาร', 'กลาง', 'หรือ', 'แม้', 'แต่', 'ผู้', 'คุม', 'ระบบ', 'แม้', 'แต่', 'คน', 'เดียว', 'เครือข่าย', 'เป็น', 'แบบ', 'เพียร์', 'ทูเพียร์', 'และ', 'การ', 'ซื้อขาย', 'เกิด', 'ขึ้น', 'ระหว่าง', 'จุด', 'ต่อ', 'เครือข่าย', '(', 'network', 'node', ')', 'โดย', 'ตรง', 'ผ่าน', 'การ', 'ใช้', 'วิทยาการ', 'เข้า', 'รหัส', 'ลับ', 'และ', 'ไม่', 'มี', 'สื่อ', 'กลาง', 'การ', 'ซื้อขาย', 'เหล่า', 'นี้', 'ถูก', 'ตรวจสอบ', 'โดย', 'รายการ', 'เดิน', 'บัญชี', 'แบบ', 'สาธารณะ', 'ที่', 'เรียก', 'ว่า', 'บล็อก', 'เชน', 'บิตคอยน์', 'ถูก', 'พัฒนา', 'โดย', 'คน', 'หรือ', 'กลุ่ม', 'คน', 'ภาย', 'ใต้', 'นาม', 'แฝง', 'ซาโตชิ นากาโมโตะ', 'และ', 'ถูก', 'เผยแพร่', 'ใน', 'รูปแบบ', 'ซอฟต์แวร์', 'โอเพนซอร์ซ', 'ใน'

['สี', 'สัน', 'ของ', 'งาน', 'นี้', 'คือ', 'ชาว', 'บ้าน', 'จอม', 'บึง', 'จะ', 'ออก', 'มา', 'ต้อนรับ', 'เหล่า', 'นัก', 'วิ่ง', 'ตลอด', 'เส้นทาง', 'มี', 'น้อง', 'ๆ', 'เด็ก', 'ๆ', 'มา', 'คอย', 'แจก', 'น้ำ', 'มา', 'ขอ', 'จับ', 'มือ', '(', 'ใคร', 'ไป', 'วิ่ง', 'ก็', 'พก', 'ขนม', 'ของ', 'แจก', 'ไป', 'ให้', 'เด็ก', 'ๆ', 'ด้วย', 'ก็', 'ดี', 'นะ', ')', 'ไป', 'จนถึง', 'ดนตรี', 'สด', 'จาก', 'ชาว', 'บ้าน', 'หลาย', 'เวที', 'เลย', 'ทีเดียว', 'จอม', 'บึง', 'มา', 'ราธอน', 'ไม่', 'มี', 'เวลา', 'Cut', 'Off', 'ครับ', 'คือ', 'เค้า', 'จะ', 'รอ', 'จนกว่า', 'คน', 'สุดท้าย', 'จะ', 'เข้า', 'เส้น', 'ชัย', 'นั่น', 'เอง']
['นัก', 'วิ่ง', 'ทั้งหลาย', 'ควร', 'มา', 'วิ่ง', 'งาน', 'จอม', 'บึง', 'มาราธอน', 'สัก', 'ครั้ง', 'ใน', 'ชีวิต']
['2', ')', 'ศิริราช', 'เดิน', '-', 'วิ่ง', 'ผสาน', 'ชุมชน', 'งาน', 'วิ่ง', 'ที่', 'จัด', 'โดย', 'โรงพยาบาลศิริราช', 'ถึงแม้', 'จะ', 'มี', 'ผู้', 'สมัคร', 'เป็น', 'จำนวน', 'มาก', 'แต่', 'ใช้', 'ฟอร์ม', 'สมัคร', 'ของ', 'Google', 'ทำ', 'ให้', 'มี', 'ความ', 'เสถียร', 'และ', 'ไม่', 'ล่มแน่นอ

['จาก', 'งาน', 'วิจัย', 'ปี', '2548', 'ใน', 'วารสาร', 'การ', 'แพทย์', 'พบ', 'ว่า', 'คน', 'เอเชีย', 'จะ', 'เริ่ม', 'มี', 'ผม', 'หงอก', 'หรือ', 'ผม', 'ขาว', 'ตั้งแต่', 'อายุ', 'ปลาย', '30', 'ปี', 'ส่วน', 'ชาว', 'ตะวัน', 'ตก', 'จะ', 'พบ', 'เส้น', 'ผม', 'เริ่ม', 'ขาว', 'ตั้งแต่', 'อายุ', '35', 'ปี', 'และ', 'ชาว', 'แอฟริกัน', '-', 'อเมริกัน', 'จะ', 'พบ', 'เส้น', 'ผม', 'ขาว', 'ได้', 'ช้า', 'กว่า', 'เมื่อ', 'อายุ', 'ประมาณ', '45', 'ปี', 'ขึ้น', 'ไป', 'สำหรับ', 'อาหาร', 'และ', 'วิตามิน', 'ที่', 'มี', 'ประโยชน์', 'บำรุง', 'ให้', 'เส้น', 'ผม', 'แข็งแรง', 'ได้แก่', 'วิตามิน', 'เอ', 'เป็น', 'สิ่ง', 'ที่', 'จำเป็น', 'ต่อ', 'สุขภาพ', 'หนัง', 'ศีรษะ', 'และ', 'เส้น', 'ผม', 'ที่', 'แข็งแรง', 'พบ', 'ทั่วไป', 'ใน', 'ผัก', 'ใบ', 'เขียว', 'เข้ม', 'และ', 'ผลไม้', 'สี', 'หลือง', 'วิตามิน', 'บี', 'ช่วย', 'ควบคุม', 'การ', 'หลั่ง', 'ของ', 'น้ำมัน', 'ธรรมชาติ', 'เพื่อ', 'หล่อลื่น', 'เส้น', 'ผม', 'ให้', 'เงางาม', 'และ', 'ชุ่มชื้น', 'พบ', 'ใน', 'ผัก', 'ใบ', 'เขียว', 'มะเขือเทศ', 'กล้วย', 'และ', 'ธัญพืช', 'และ', 'แ

['มา', 'วิน', 'ตัดสินใจ', 'หนี', 'ออก', 'จาก', 'บ้าน', 'ไป', 'ขอ', 'อาศัย', 'กับ', 'ครอบครัว', 'ทอฝัน', 'ที่', 'นั่น', 'เอง', 'มา', 'วิน', 'ได้', 'สัมผัส', 'กับ', 'ความ', 'อบอุ่น', 'ของ', 'ครอบครัว', 'อย่าง', 'แท้จริง', 'ชนะ', 'ออก', 'ตาม', 'หา', 'มา', 'วิน', 'และ', 'รู้สึก', 'ไม่', 'อยาก', 'สูญเสีย', 'ใคร', 'ไป', 'อีก', 'แล้ว', 'พอ', 'ชนะ', 'รู้', 'ว่า', 'มา', 'วิน', 'ไป', 'อยู่', 'กับ', 'ครอบครัว', 'ทอ', 'ฝัน', 'ก็', 'โมโห', 'มาก', 'รีบ', 'บึ่ง', 'รถ', 'ไป', 'บ้านทอ', 'ฝัน', 'แต่', 'ระหว่าง', 'ทาง', 'เกิด', 'อุบัติเหตุ', 'รถ', 'ชน', 'เสา', 'ไฟฟ้า', 'มา', 'วิน', 'รู้', 'ข่าว', 'ก็', 'รีบ', 'ไป', 'หา', 'พ่อ', 'ที่', 'โรง', 'พยาบาล', 'ทันที', 'ใน', 'ขณะ', 'ที่', 'ทอ', 'ฝัน', 'ก็', 'รู้สึก', 'ผิด', 'ที่', 'เป็น', 'ต้น', 'เหตุ', 'ทำ', 'ให้', 'ชนะ', 'กับ', 'มา', 'วิน', 'ทะเลาะ', 'กัน', 'ทอฝัน', 'จึง', 'ตัดสินใจ', 'ขอ', 'ยุติ', 'ความ', 'สัมพันธ์', 'กับ', 'มา', 'วิน', 'ทั้ง ', 'ๆ', 'ที่', 'รัก', 'มา', 'วิน', 'สุด', 'หัวใจ', 'สุดท้าย', 'ความ', 'รัก', 'ระหว่าง', 'ทอ', 'ฝัน', 'กับ', 'มา', 'วิน'

In [5]:
model = Word2Vec(document_data, size=100, window=3, min_count=0)

In [222]:
numeric_question_keywords = ['กี่', 'เท่าไหร่', 'เท่าใด', 'เท่าไร']
location_question_keywords = ['ที่ใด', 'แห่งใด', 'ที่ไหน', 'บริเวณใด', 'จังหวัดใด']
person_question_keywords = ['คือใคร', 'ใครเป็น', 'ใคร', 'ผู้ใด', 'ชนชาติใด', 'ชื่ออะไร']

numeric_question_time_keywords = ['นาน', 'อายุ', 'เวลา']
numeric_question_money_keywords = ['เงิน', 'ราคา']
numeric_question_quantity_keywords = ['ยาว']

person_clue_keywords = ['โดย']

numeric_time_unit_keywords = ['วัน', 'เดือน', 'ปี']
numeric_money_unit_keywords = ['บาท', 'ดอลลาร์']
numeric_quantity_unit_keywords = ['เมตร', 'นิ้ว', 'ฟุต', 'หลา', 'ไมล์']
location_answer_keywords = ['โครง', 'โรง', 'จังหวัด', 'อำเภอ', 'ตำบล', 'ประเทศ', 'วัด', 'ทวีป', 'สถานี', 'เขต', 'พื้นที่', 'สนามบิน']
person_answer_keywords = ['นาย', 'นาง', 'สม', 'น.ส.', 'ด.ช.', 'เด็กชาย', 'ด.ญ.', 'เด็กหญิง', 'คุณ']
person_all_rank_keywords = ['หม่อม', 'สมเด็จเจ้า', 'เจ้าพระยา', 'พระยา', 'พระ', 'จมื่น', 'หลวง', 'ขุน', 'หมื่น', 'พัน', 'นาย', 'หมู่', 'พระนาย', 'ออกญา', 'ออกพญา', 'ออกพระ', 'ออกขุน', 'ออกหลวง', 'พญา', 'ท้าว', 'มหาเทวี', 'เทวี']
person_gov_rank_keywords = ['จอมพล', 'จอมพลเรือ', 'จอมพลอากาศ', 'พลเอก', 'พลเรือเอก', 'พลอากาศเอก', 'พลโท', 'พลเรือโท', 'พลอากาศโท', 'พลตรี', 'พลเรือตรี', 'พลอากาศตรี', 'พลจัตวา', 'พลเรือจัตวา', 'พลอากาศจัตวา', 'พันเอก', 'นาวาเอก', 'นาวาอากาศเอก', 'พันโท', 'นาวาโท', 'นาวาอากาศโท', 'พันตรี', 'นาวาตรี', 'นาวาอากาศตรี', 'ร้อยเอก', 'เรือเอก', 'เรืออากาศเอก', 'ร้อยโท', 'เรือโท', 'เรืออากาศโท', 'ร้อยตรี', 'เรือตรี', 'เรืออากาศตรี', 'จ่าสิบเอก', 'พันจ่าเอก', 'พันจ่าอากาศเอก', 'จ่าสิบโท', 'พันจ่าโท', 'พันจ่าอากาศโท', 'จ่าสิบตรี', 'พันจ่าตรี', 'พันจ่าอากาศตรี', 'สิบเอก', 'จ่าเอก', 'จ่าอากาศเอก', 'สิบโท', 'จ่าโท', 'จ่าอากาศโท', 'สิบตรี', 'จ่าตรี', 'จ่าอากาศตรี', 'พลทหาร', 'พล.อ.', 'พล.ท.', 'พล.ต.', 'พล.จ.', 'พ.อ.', 'พ.ท.', 'พ.ต.', 'ร.อ.', 'ร.ท.', 'ร.ต.', 'นนร.', 'จ.ส.อ(พ.).', 'จ.ส.อ.', 'จ.ส.ท.', 'จ.ส.ต.', 'ส.อ.', 'ส.ท.', 'ส.ต.', 'ส.ต.', 'พลฯ', 'พล.ร.อ.', 'พล.ร.ท.', 'พล.ร.ต.', 'พล.ร.จ.', 'น.อ.', 'น.ท.', 'น.ต.', 'ร.อ.', 'ร.ท.', 'ร.ต.', 'พ.จ.อ.(พ.)', 'พ.จ.อ.', 'พ.จ.ท.', 'พ.จ.ต.', 'จ.อ.', 'จ.ท.', 'จ.ต.', '', 'ล.อ.อ.', 'พล.อ.ท.', 'พล.อ.ต.', 'พล.อ.จ.', 'น.อ.', 'น.ท.', 'น.ต.', 'ร.อ.', 'ร.ท.', 'ร.ต.', 'พ.อ.อ.(พ.)', 'พ.อ.อ.', 'พ.อ.ท.', 'พ.อ.ต.', 'จ.อ.', 'จ.ท.', 'จ.ต.', 'พล.ต.อ.', 'พล.ต.ท.', 'พล.ต.ต.', 'พล.ต.จ.', 'พ.ต.อ.', 'พ.ต.ท.', 'พ.ต.ต.', 'ร.ต.อ.', 'ร.ต.ท.', 'ร.ต.ต.', 'นรต.']

KEE_question_index_list = [1, 3, 4, 5, 6, 8]
this_question = question_list[10]
question_num = this_question[0]
question_text = this_question[1]

print(question_text)

question_type = ''
question_keyword = ''

for keyword in numeric_question_keywords:
    if question_text.__contains__(keyword):
        question_type = 'numeric'
        question_keyword = keyword
        break
if not question_type:
    for keyword in location_question_keywords:
        if question_text.__contains__(keyword):
            question_type = 'location'
            question_keyword = keyword
            break
if not question_type:
    for keyword in person_question_keywords:
        if question_text.__contains__(keyword):
            question_type = 'person'
            question_keyword = keyword
            break

tokenized_question_text = word_tokenize(text=question_text, engine='deepcut')
tokenized_question_text = [x for x in tokenized_question_text if x != ' ']
# question_text = question_text.replace(question_keyword, '')
print(tokenized_question_text)
print(question_type, question_keyword)

if question_keyword == 'กี่':
    unit_keyword = tokenized_question_text[tokenized_question_text.index(question_keyword) + 1]
    print(question_keyword, unit_keyword)

ใครเป็นผู้กำกับการแสดงละครเรื่องตะวันยอแสงของช่อง 3  ในปี พ.ศ. 2560
['ใคร', 'เป็น', 'ผู้', 'กำกับ', 'การ', 'แสดง', 'ละคร', 'เรื่อง', 'ตะวัน', 'ยอแสง', 'ของ', 'ช่อง', ' 3', 'ใน', 'ปี', 'พ.ศ.', '2560']
person ใครเป็น


In [225]:
found_paragraph_list = []
for paragraph in document_data:
    # print(paragraph)
    for question_word in tokenized_question_text:
        matching = [s for s in paragraph if question_word in s]
        if matching:
            found_paragraph_list.append(paragraph)
            # print(matching)

unique_found_paragraph = [list(x) for x in set(tuple(x) for x in found_paragraph_list)]

filtered_tokenized_question_text = [word for word in tokenized_question_text if word not in thai_stopwords]
not_thai_word_paragraph_list = []

co_occur = [[0 for x in range(len(tokenized_question_text))] for y in range(len(unique_found_paragraph))] 
for p_idx, para in enumerate(unique_found_paragraph):
    for p_word in para:
        for q_idx, q_word in enumerate(tokenized_question_text):
            if q_word == p_word:
#                 print('[{}][{}] {} {}'.format(p_idx, q_idx, q_word, p_word))
                if co_occur[p_idx][q_idx] == 0:
                    co_occur[p_idx][q_idx] = 1
            if isthai(q_word,check_all=False)['thai'] == 0.0 and not any(i.isdigit() for i in q_word):
                if q_word == p_word:
                    not_thai_word_paragraph_list.append(para)
            
print("Q", tokenized_question_text, "\n---------------------------------------------------------------")
sum_co_occur = []
for c_idx, c1 in enumerate(co_occur):
    sum_co_occur.append(sum(c1))
#     print(c1, '=', sum(c1)) #unique_found_paragraph[c_idx]

top_co_occur_idx = sorted(range(len(sum_co_occur)), key=lambda i: sum_co_occur[i])[-5:]
print(top_co_occur_idx)
for idx in top_co_occur_idx:
    print(sum_co_occur[idx], "->", unique_found_paragraph[idx])
    print('====================================================================')
# max_co_occur_idx = sum_co_occur.index(max(sum_co_occur))
# print(sum_co_occur[max_co_occur_idx], "->", unique_found_paragraph[max_co_occur_idx])
print(not_thai_word_paragraph_list)

Q ['ใคร', 'เป็น', 'ผู้', 'กำกับ', 'การ', 'แสดง', 'ละคร', 'เรื่อง', 'ตะวัน', 'ยอแสง', 'ของ', 'ช่อง', ' 3', 'ใน', 'ปี', 'พ.ศ.', '2560'] 
---------------------------------------------------------------
[143, 161, 11, 54, 51]
9 -> ['ครั้ง', 'ที่', '4', 'ใน', 'ปี', 'พ.ศ.', '2560', 'ออก', 'อากาศ', 'ทาง', 'สถานีวิทยุโทรทัศน์', 'ไทย', 'ทีวี', 'สี', 'ช่อง', ' 3', 'สร้าง', 'โดย', 'บริษัท เมตตา', 'และ', 'มหานิยม จำกัด', 'ควบคุม', 'การ', 'ดำเนิน', 'งาน', 'โดย', 'สินจัย', 'เปล่งพานิช', 'บท', 'โทรทัศน์', ' คฑาหัสต์', ' บุษปะเกศ', 'กำกับ', 'การ', 'แสดง', 'โดย', 'โชติรัตน์', 'รักเริ่ม', 'วงศ์', 'นำ', 'แสดง', 'โดย', 'ณัฐวุฒิ สกิดใจ', ',', 'พิจักขณา วงศารัตนศิลป์', ',', 'อรรณพ ทองบริสุทธิ์', ',', 'วชิรวิชญ์', 'ไพศาลกุลวงศ์', ',', 'อริสรา', ' ทองบริสุทธิ์', ',', 'รฐา โพธิ์งาม', ',', 'ฉัตรชัย เปล่งพานิช', ',', 'อาภาศิริ นิติพน']
9 -> ['ถูก', 'นำ', 'ไป', 'สร้าง', 'เป็น', 'ละคร', 'โทรทัศน์', 'มา', 'แล้ว', '3', 'ครั้ง', 'ใน', 'ปี', 'พ.ศ.', '2521', 'และ', 'ใน', 'ปี', 'พ.ศ.', '2540', 'ออก', 'อากาศ', 'ทาง', 'สถ

In [218]:
print(question_num, tokenized_question_text, "\n---------------------------------------------------------------")
answer_choice_list = []
if question_keyword == 'กี่':
    for idx in top_co_occur_idx:
        focused_paragraph = unique_found_paragraph[idx]
        first_keyword = tokenized_question_text[tokenized_question_text.index(tokenized_question_text[0])]
        last_keyword = tokenized_question_text[tokenized_question_text.index(question_keyword) - 1]
        unit_keyword = tokenized_question_text[tokenized_question_text.index(question_keyword) + 1]
        print(first_keyword, " -> ", last_keyword, question_keyword, unit_keyword)
#         print("first_idx", tokenized_question_text.index(tokenized_question_text[0]))
#         print("last_idx", tokenized_question_text.index(question_keyword) - 1)
#         print("unit_idx", tokenized_question_text.index(question_keyword) + 1)
        try:    
            first_idx = focused_paragraph.index(first_keyword)
        except ValueError:
            first_idx = -1
        try:
            last_idx = focused_paragraph.index(last_keyword) + 1
        except ValueError:
            last_idx = -1
        try:
            last_idx = focused_paragraph.index(tokenized_question_text[tokenized_question_text.index(question_keyword) - 2]) + 0
        except ValueError:
            last_idx = -1
        try:
            unit_idx = focused_paragraph.index(unit_keyword)
            normal_flow = True
        except ValueError:
            normal_flow = False
        if (normal_flow):
            print(focused_paragraph[first_idx:last_idx+2])
            direct_answer = focused_paragraph[last_idx:unit_idx+1]
            number_in_direct_answer = re.findall(r'\b\d+\b', ' '.join(direct_answer))
            if number_in_direct_answer:
                actual_direct_unit = focused_paragraph[focused_paragraph.index(number_in_direct_answer[0]) + 1]
                if actual_direct_unit == unit_keyword:
                    print(number_in_direct_answer[0], actual_direct_unit)
                    answer_choice_list.append('{} {}'.format(number_in_direct_answer[0], unit_keyword))
            unit_indices = [i for i, x in enumerate(focused_paragraph) if x == unit_keyword]
            next_first_idx = first_idx
            print("-----------------------------------")
            for u_idx in unit_indices:
                cut_paragraph = focused_paragraph[next_first_idx:u_idx+1]
                number_in_cut_paragraph = re.findall(r'\b\d+\b', ' '.join(cut_paragraph))
                for number_text in number_in_cut_paragraph:
                    num_idx  = cut_paragraph.index(number_text)
                    answer_number = cut_paragraph[num_idx]
                    answer_unit = cut_paragraph[num_idx+1]
                    print("@", answer_number, answer_unit)
                    if answer_unit == "-":
                        answer_number_text = ''.join(cut_paragraph[num_idx:num_idx+3])
                        answer_unit = cut_paragraph[num_idx+3]
                        if answer_unit == unit_keyword:
                            answer_text = '{} {}'.format(answer_number_text, answer_unit)
                            print("answer_text", answer_text)
                            answer_choice_list.append(answer_text)
                    elif answer_unit == unit_keyword:
                        if all(i.isdigit() for i in answer_unit):
                            if float(answer_unit) != 0:
                                answer_text = '{} {}'.format(answer_number, answer_unit)
                                answer_choice_list.append(answer_text)
                                print('&', answer_number, answer_unit)
                                print("answer_choice_list", answer_choice_list)
                                print("------------------------------------------")
                        else:
                            if all(i.isdigit() for i in answer_number):
                                if float(answer_number) != 0:
                                    answer_text = '{} {}'.format(answer_number, answer_unit)
                                    answer_choice_list.append(answer_text)
                                    print('^', answer_number, answer_unit)
                                    print("answer_choice_list", answer_choice_list)
                                    print("------------------------------------------")
                print(cut_paragraph, '=>', number_in_cut_paragraph)
                next_first_idx = u_idx+1
        if (not normal_flow or not answer_choice_list):
            for para_eng in not_thai_word_paragraph_list:
                unit_indices = [i for i, x in enumerate(para_eng) if x == unit_keyword]
                for u_idx in unit_indices:
                    direct_answer_number = para_eng[u_idx-1]
                    direct_answer_unit = para_eng[u_idx]
                    if any(i == '-' for i in direct_answer_number):
                        direct_answer_number = para_eng[u_idx-1:u_idx+3]
                        direct_answer_unit = para_eng[u_idx+3]
                    if all(i.isdigit() for i in direct_answer_number):
                        answer_choice_list.append('{} {}'.format(direct_answer_number, direct_answer_unit))
                        print("direct_answer_number, direct_answer_unit", direct_answer_number, direct_answer_unit)
        print("===================================")
elif question_type == 'numeric' and question_keyword != 'กี่':
    numeric_unit_list = []
    for keyword in numeric_question_time_keywords:
        if question_text.__contains__(keyword):
            numeric_unit_list = numeric_time_unit_keywords
            break
    if not numeric_unit_list:
        for keyword in numeric_question_money_keywords:
            if question_text.__contains__(keyword):
                numeric_unit_list = numeric_money_unit_keywords
                break
    if not numeric_unit_list:
        for keyword in numeric_question_quantity_keywords:
            if question_text.__contains__(keyword):
                numeric_unit_list = numeric_quantity_unit_keywords
                break
    for idx in top_co_occur_idx:
        focused_paragraph = unique_found_paragraph[idx]
        number_in_focused_paragraph = re.findall(r'\b\d+\b', ' '.join(focused_paragraph))
        print("number_in_focused_paragraph", number_in_focused_paragraph)
        for number_text in number_in_focused_paragraph:
            number_idx = focused_paragraph.index(number_text)
            unit_idx = number_idx + 1
            number_text = focused_paragraph[number_idx]
            unit_text = focused_paragraph[unit_idx]
            print(number_text, unit_text)
            if numeric_unit_list == numeric_money_unit_keywords:
                if unit_text == ',':
                    next_number_text = focused_paragraph[unit_idx+1]
                    if all(i.isdigit() for i in next_number_text):
                        number_text = ''.join(focused_paragraph[number_idx:number_idx+3])
                        unit_text = focused_paragraph[number_idx+3]
                        print("$", number_text, unit_text)
                        candidate_answer_text = '{} {}'.format(number_text, unit_text)
                        answer_choice_list.append(candidate_answer_text)
#             elif (not answer_choice_list) or (unit_text in numeric_unit_list):
            else:
#                 print(number_text, unit_text, "in")
                check_exist_in_unit_list = []
                for u_word in numeric_unit_list:
                    if u_word in unit_text:
                        check_exist_in_unit_list.append(u_word)
                        break
                if check_exist_in_unit_list:
                    print(unit_text, "in?", numeric_unit_list)
                    if (float(number_text) != 0) and (check_exist_in_unit_list):
                        candidate_answer_text = '{} {}'.format(number_text, unit_text)
                        answer_choice_list.append(candidate_answer_text)
                        print(keyword, '=>', candidate_answer_text)
        #                 print(number_in_focused_paragraph)
        print("===================================")

Q11 ['ใคร', 'เป็น', 'ผู้', 'กำกับ', 'การ', 'แสดง', 'ละคร', 'เรื่อง', 'ตะวัน', 'ยอแสง', 'ของ', 'ช่อง', ' 3', 'ใน', 'ปี', 'พ.ศ.', '2560'] 
---------------------------------------------------------------


In [219]:
def unique_list(seq):
    seen = set()
    seen_add = seen.add
    return [x for x in seq if not (x in seen or seen_add(x))]

answer_string = '{}::'.format(question_num)
answer_choice_list = unique_list(answer_choice_list)
for idx, ans in enumerate(answer_choice_list):
    if (idx == 5):
        break
    answer_string = answer_string + ans + '||'
answer_string = answer_string[:-2]
print(answer_string)

Q11


In [10]:
aa=model.similar_by_word('เป็น')
print(aa)
# aa=model['เป็น']
# print(aa)
print(model.similarity('รถ', 'รถ'))
# print(model['รถ'], model['ยนต์'])

[('และ', 0.9999482035636902), ('ที่', 0.9999467730522156), ('ใน', 0.999945342540741), ('โดย', 0.9999383687973022), ('ได้', 0.9999353885650635), ('หรือ', 0.9999293088912964), ('มี', 0.9999291300773621), ('จะ', 0.9999282360076904), ('ของ', 0.9999244213104248), ('(', 0.999923825263977)]
1.0


C:\Users\Watchanan\PycharmProjects\thai-question-answering\venv\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similar_by_word` (Method will be removed in 4.0.0, use self.wv.similar_by_word() instead).
  """Entry point for launching an IPython kernel.
C:\Users\Watchanan\PycharmProjects\thai-question-answering\venv\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """


In [11]:
vector_of_question_text = 0
for word in tokenized_question_text:
    vector_of_question_text = vector_of_question_text + sum(model[word])
print("vector_of_question_text", vector_of_question_text)

filtered_tokenized_question_text = [word for word in tokenized_question_text if word not in stopwords]

found_idx = set()
for idx, para in enumerate(unique_found_paragraph):
    filtered_words = [word for word in para if word not in stopwords]
    for p_word in filtered_words:
        for q_word in filtered_tokenized_question_text:
            sim = model.similarity(p_word, q_word)
            if sim >= 0.9:
                found_idx.add(idx)
                print(p_word, '&', q_word, sim)

print(sorted(list(found_idx)))

vector_of_question_text 4.796636071201647


C:\Users\Watchanan\PycharmProjects\thai-question-answering\venv\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


TypeError: argument of type 'module' is not iterable